<a href="https://colab.research.google.com/github/nicolerdz/Regresion-no-lineal-y-logistica/blob/main/2020_9_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar archivo csv desde seaborn
df2020=pd.read_csv('Gastos_y_costos_2020_sin_nulos.csv', index_col= 0)
df2020

,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,GASTO,DESCRIPCION,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,TIPO,STATUS,POLIZA,Columna1
0,2020-01-01,39408,BD20F436-72CC-4BE2-AD37-5999D71AA391,ARE080115PBA,ARSI RENTA SA DE CV,MAQUINARIA,0,"Descripción: REVOLVEDORA, Modelo: ULTRAMP9A, M...",1.0,0.00,2248.28,0.0,0.0,2248.28,16300.00,I,Vigente,SIN POLIZA,14051.72
1,2020-01-01,0001010100750847116902,85AEAC2E-6395-44A0-9CB9-754D06893375,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,0,COMISION COBRADA,1.0,490.00,78.40,0.0,0.0,568.40,568.40,I,Vigente,SIN POLIZA,0.00
2,2020-01-01,0001010100752163714102,BD0DFE85-D776-41CB-9C5C-4FA7923E687A,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,0,COMISION COBRADA,1.0,195.00,31.20,0.0,0.0,226.20,226.20,I,Vigente,SIN POLIZA,0.00
3,2020-01-01,3663,EF4504D6-830D-415C-AF03-80CF54C5E28E,PSS120625C86,PROMOTORA DE SERVICIOS Y SOLUCIONES EMPRESARIA...,HONORARIOS CONTABLES,0,HONORARIOS CONTABLES\nDEL MES DE ENERO DEL 2020.,1.0,10694.25,1711.08,0.0,0.0,12405.33,12405.33,I,Vigente,SIN POLIZA,0.00
4,2020-01-01,2849,EC2F4AA4-BA64-4E2E-A5B7-4EBBE042D6D3,ENA7006081P0,ELECTROTERMICA NACIONAL SA DE CV,COMPRAS,0,Pago,1.0,0.00,519.80,0.0,0.0,0.00,0.00,P,Vigente,SIN POLIZA,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,2020-12-01,125834,80BCB1D8-673F-495B-9C2A-4AC9F9CB38E5,SQS110113MU0,SERVICIO QUINTANA SUR S.A DE C.V,COMBUSTIBLE,G VTA,32011 MAGNA,1.0,129.79,20.21,0.0,0.0,150.00,150.00,I,Vigente,SIN POLIZA,0.00
3338,2020-12-01,37652,D4C7C1FB-03A7-4046-A2BD-03EBE7DA861E,PSQ790724C90,"CIA. PERIODISTICA DEL SOL DE QUERETARO, S.A. D...",PUBLICIDAD,G VTA,82101504-PUBLICIDAD EN PERIODICOSCALOR Y CONTROL,1.0,2284.80,365.57,0.0,0.0,2650.37,2650.37,I,Vigente,SIN POLIZA,0.00
3339,2020-12-01,160,FFCEE7D4-1F38-4D44-BE02-EB042DAACEE7,SQX981027RY5,"SUPER Q, S. A. DE C. V.",ALIMENTOS,G VTA,REF VICTORIA GROSELLA 3 LT PET NR,1.0,163.79,26.21,0.0,0.0,190.00,190.00,I,Vigente,SIN POLIZA,0.00
3340,2020-12-01,634875,8DFF4F01-2F8B-4085-B116-05A314108B02,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",COMISION BANCARIA,G FINA,COMISIONES E INTERESES,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,SIN POLIZA,0.00


In [ ]:
#Rellenamos nulos
df2020 =df2020.fillna(method="bfill")
df2020 =df2020.fillna(method="ffill")


## **Reemplazar valores**

In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df2020['TIPO GASTO'])
unico

array(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'ATENCION CLIENTES', 'CALIBRACIONES', 'CAPACITACION',
       'COMBUSTIBLE', 'COMISION BANCARIA', 'COMISION VENTA', 'COMPRAS',
       'DERECHOS', 'ENERGIA ELECTRICA', 'FLETES', 'GERENCIA ADMON',
       'GERENCIA VTAS', 'HONORARIOS ADMON', 'HONORARIOS CONTABLES',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT', 'MAQUILAS GIC',
       'MAQUINARIA', 'MEDICOS', 'MENSAJERIA', 'MOBILIARIO',
       'MTTO COMPUTO', 'MTTO LOCAL', 'MTTO TRANSPORTE', 'NO DEDUCIBLE',
       'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'REPARACIONES ELECTRICAS', 'SEGUROS', 'SERV FUNERARIO ',
       'SUBCONTRATOS', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORME', 'VALES DESPENSA'], dtype=object)

In [ ]:
#Convierto una variable a dicotómica 
df2020['TIPO GASTO']= df2020['TIPO GASTO'].replace(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'ATENCION CLIENTES', 'CALIBRACIONES', 'CAPACITACION',
       'COMBUSTIBLE', 'COMISION BANCARIA', 'COMISION VENTA',
       'DERECHOS', 'ENERGIA ELECTRICA', 'FLETES', 'GERENCIA ADMON',
       'GERENCIA VTAS', 'HONORARIOS ADMON', 'HONORARIOS CONTABLES',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT', 'MAQUILAS GIC',
       'MAQUINARIA', 'MEDICOS', 'MENSAJERIA', 'MOBILIARIO',
       'MTTO COMPUTO', 'MTTO LOCAL', 'MTTO TRANSPORTE', 'NO DEDUCIBLE',
       'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'REPARACIONES ELECTRICAS', 'SEGUROS', 'SERV FUNERARIO ',
       'SUBCONTRATOS', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORME', 'VALES DESPENSA'], "OTROS GASTOS")
df2020

,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,GASTO,DESCRIPCION,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,TIPO,STATUS,POLIZA,Columna1
0,2020-01-01,39408,BD20F436-72CC-4BE2-AD37-5999D71AA391,ARE080115PBA,ARSI RENTA SA DE CV,OTROS GASTOS,0,"Descripción: REVOLVEDORA, Modelo: ULTRAMP9A, M...",1.0,0.00,2248.28,0.0,0.0,2248.28,16300.00,I,Vigente,SIN POLIZA,14051.72
1,2020-01-01,0001010100750847116902,85AEAC2E-6395-44A0-9CB9-754D06893375,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,0,COMISION COBRADA,1.0,490.00,78.40,0.0,0.0,568.40,568.40,I,Vigente,SIN POLIZA,0.00
2,2020-01-01,0001010100752163714102,BD0DFE85-D776-41CB-9C5C-4FA7923E687A,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,0,COMISION COBRADA,1.0,195.00,31.20,0.0,0.0,226.20,226.20,I,Vigente,SIN POLIZA,0.00
3,2020-01-01,3663,EF4504D6-830D-415C-AF03-80CF54C5E28E,PSS120625C86,PROMOTORA DE SERVICIOS Y SOLUCIONES EMPRESARIA...,OTROS GASTOS,0,HONORARIOS CONTABLES\nDEL MES DE ENERO DEL 2020.,1.0,10694.25,1711.08,0.0,0.0,12405.33,12405.33,I,Vigente,SIN POLIZA,0.00
4,2020-01-01,2849,EC2F4AA4-BA64-4E2E-A5B7-4EBBE042D6D3,ENA7006081P0,ELECTROTERMICA NACIONAL SA DE CV,COMPRAS,0,Pago,1.0,0.00,519.80,0.0,0.0,0.00,0.00,P,Vigente,SIN POLIZA,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,2020-12-01,125834,80BCB1D8-673F-495B-9C2A-4AC9F9CB38E5,SQS110113MU0,SERVICIO QUINTANA SUR S.A DE C.V,OTROS GASTOS,G VTA,32011 MAGNA,1.0,129.79,20.21,0.0,0.0,150.00,150.00,I,Vigente,SIN POLIZA,0.00
3338,2020-12-01,37652,D4C7C1FB-03A7-4046-A2BD-03EBE7DA861E,PSQ790724C90,"CIA. PERIODISTICA DEL SOL DE QUERETARO, S.A. D...",OTROS GASTOS,G VTA,82101504-PUBLICIDAD EN PERIODICOSCALOR Y CONTROL,1.0,2284.80,365.57,0.0,0.0,2650.37,2650.37,I,Vigente,SIN POLIZA,0.00
3339,2020-12-01,160,FFCEE7D4-1F38-4D44-BE02-EB042DAACEE7,SQX981027RY5,"SUPER Q, S. A. DE C. V.",OTROS GASTOS,G VTA,REF VICTORIA GROSELLA 3 LT PET NR,1.0,163.79,26.21,0.0,0.0,190.00,190.00,I,Vigente,SIN POLIZA,0.00
3340,2020-12-01,634875,8DFF4F01-2F8B-4085-B116-05A314108B02,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",OTROS GASTOS,G FINA,COMISIONES E INTERESES,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,SIN POLIZA,0.00


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df2020[['TOTAL MX']]
Var_Dep= df2020['TIPO GASTO']

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS', ...,
       'OTROS GASTOS', 'OTROS GASTOS', 'OTROS GASTOS'], dtype=object)

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0 428]
 [  1 574]]


In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="weighted", pos_label="OTROS GASTOS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.3284059995661717


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS GASTOS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5722831505483549


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="weighted", pos_label="OTROS GASTOS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5722831505483549


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS GASTOS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="weighted", pos_label="OTROS GASTOS")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.4173275986877667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS GASTOS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
